# Proof of Concept: Archi to Neo4j to App

## Background

I often use Archi for my daily work. During my research I came across the following article: https://medium.com/geekculture/using-a-graph-database-to-explore-your-archimate-model-df7bd63f65dd. The aim of this PoC is to reconstruct the described procedure and to solve a use case with it.

## Setting the Stage

As a basis, I have created an exemplary reference model with Archi, which containts simplified elements of APQC'S PROCESS CLASSIFICATION FRAMEWORK® CROSS-INDUSTRY.


